## Testing LLM and Spotify API

In [77]:
##Imports
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from transformers import pipeline
from langchain.llms import HuggingFaceHub

In [78]:
##Functions
def get_song_name(response : str) -> str:
    # Process the response to extract only the song name
    # Assuming the format is like: '"Out of Love" is a song by Avicii.'
    start_quote_index = response.find('"')  # Find the index of the first quote
    end_quote_index = response.find('"', start_quote_index + 1)  # Find the index of the closing quote

    if start_quote_index != -1 and end_quote_index != -1:
        # Extract the song name using slicing
        song_name = response[start_quote_index + 1:end_quote_index]
        return song_name
    else:
        return "Song name not found."
    
def get_track_from_spotify(spotify_client_id, spotify_client_secret, song_name : str) -> str:
    auth_manager = SpotifyClientCredentials(client_id=spotify_client_id, client_secret=spotify_client_secret)
    sp = spotipy.Spotify(auth_manager=auth_manager)
    # Search for the song
    results = sp.search(q=song_name, type='track', limit=1)

    # Extract track information
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        return(f"Found track: {track['name']} by {track['artists'][0]['name']}")
    else:
        return("Song not found.")
    
def prompt_llm(prompt : str, prompt_template : PromptTemplate = PromptTemplate()) -> str:
    llm = HuggingFaceHub(
    repo_id="ibm-granite/granite-3.0-3b-a800m-instruct",  # replace with the model name you want
    huggingfacehub_api_token=os.getenv('HUGGING_FACE_TOKEN')
    )
    if prompt_template:
        llm_chain = LLMChain(llm= llm, prompt= prompt_template)
        response = llm_chain.run(prompt)
    else:
        response = llm(prompt)

    return response

In [79]:

# Replace these with your actual Spotify API credentials
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')

In [80]:
#Langchain model

llm = HuggingFaceHub(
    repo_id="ibm-granite/granite-3.0-3b-a800m-instruct",  # replace with the model name you want
    huggingfacehub_api_token=os.getenv('HUGGING_FACE_TOKEN')
)

# Generate text with the LangChain LLM
response = llm("Recommend me a song from Metallica. But include in your response only the name of the song, and only one song.")
#print(response)

song_name = get_song_name(response)

In [81]:
print(get_track_from_spotify(client_id, client_secret, song_name))

Found track: Enter Sandman (Remastered) by Metallica


## Using promt template

In [82]:
song_description = 'Instrumental music with a piano in the background'

#Templates
song_template = PromptTemplate(
    input_variables=['description'],
    template='Recommend me a song that matches this description: {song_description}.\
        Include in your response only the name of the song, and only one song.'
)

    
song_name = get_song_name(prompt_llm(song_template, song_description))

In [83]:
print(get_track_from_spotify(client_id, client_secret, song_name))

Found track: Clair de lune by Claude Debussy
